# Install and load necesary packages

In [1]:
# Please don't change this cell

import pandas as pd
import numpy as np  

import warnings
warnings.filterwarnings("ignore")

## Load the dataset using pandas

In [2]:
# Please don't change this cell
df = pd.read_csv('ml-100k/u.data', names=['user_id', 'item_id', 'rating', 'timestamp'], sep='\t')

df.head()

,user_id,item_id,rating,timestamp
0,196,242,3,881250949
1,186,302,3,891717742
2,22,377,1,878887116
3,244,51,2,880606923
4,166,346,1,886397596


# Split dataset


## Randomly select one rating from each user as test set

In [3]:
# please do not change this cell

from sklearn.model_selection import train_test_split

n_users = df.user_id.unique().shape[0]
n_items = df.item_id.unique().shape[0]
print(str(n_users) + ' users')
print(str(n_items) + ' items')

train_df, test_df = train_test_split(df, test_size=0.2, random_state = 10)
train_df, test_df

# Training Dataset
train_ds = np.zeros((n_users, n_items))
item_popularity = np.zeros(n_items)
for row in train_df.itertuples():
    train_ds[row[1]-1, row[2]-1] = row[3]
    item_popularity[row[2]-1] =  item_popularity[row[2]-1] + 1
#train_ds = pd.DataFrame(train_ds)

# Testing Dataset
testsize = 0
test_ds = np.zeros((n_users, n_items))
for row in test_df.itertuples():
    if item_popularity[row[2]-1] > 30:
        test_ds[row[1]-1, row[2]-1] = row[3]
        testsize = testsize + 1
#test_ds = pd.DataFrame(test_ds)

print("Construct the rating matrix based on train_df:")
print(train_ds)

print("Construct the rating matrix based on test_df:")
print(test_ds)

print("Testsize = " + str(testsize))

943 users
1682 items
Construct the rating matrix based on train_df:
[[0. 3. 4. ... 0. 0. 0.]
 [4. 0. 0. ... 0. 0. 0.]
 [0. 0. 0. ... 0. 0. 0.]
 ...
 [5. 0. 0. ... 0. 0. 0.]
 [0. 0. 0. ... 0. 0. 0.]
 [0. 5. 0. ... 0. 0. 0.]]
Construct the rating matrix based on test_df:
[[5. 0. 0. ... 0. 0. 0.]
 [0. 0. 0. ... 0. 0. 0.]
 [0. 0. 0. ... 0. 0. 0.]
 ...
 [0. 0. 0. ... 0. 0. 0.]
 [0. 0. 0. ... 0. 0. 0.]
 [0. 0. 0. ... 0. 0. 0.]]
Testsize = 17678


# MAE and RMSE Utils

In [4]:
# Please don't change this cell
# you can use this devaluate Utils here, and you can also implement your own MAE and RMSE calculation. 

EPSILON = 1e-9

def evaluate(test_ds, predicted_ds):
    '''
    Function for evaluating on MAE and RMSE
    '''
    # MAE
    mask_test_ds = test_ds > 0
    MAE = np.sum(np.abs(test_ds[mask_test_ds] - predicted_ds[mask_test_ds])) / np.sum(mask_test_ds.astype(np.float32))

    # RMSE
    RMSE = np.sqrt(np.sum(np.square(test_ds[mask_test_ds] - predicted_ds[mask_test_ds])) / np.sum(mask_test_ds.astype(np.float32)))

    return MAE, RMSE

# Your Solution for Method 1

In [5]:
# Write your code here for Method 1
# You are required to implement the required solution 1 here. 
# Then, evaluate your implementation by predicting the ratings in the test set (test_ds).
# Finally, save the corresponding MAE and RMSE of your implementation 
# into the following defined corresponding variable. 


# Setting up parameters
ratings = train_ds
userRatedCount = (ratings > 0).sum(axis=1)
userRatedSum = ratings.sum(axis=1)
userAvg = userRatedSum/userRatedCount


i_cols = ['movie id', 'movie title' ,'release date','video release date', 'IMDb URL', 'unknown', 'Action', 'Adventure',
 'Animation', 'Children\'s', 'Comedy', 'Crime', 'Documentary', 'Drama', 'Fantasy',
 'Film-Noir', 'Horror', 'Musical', 'Mystery', 'Romance', 'Sci-Fi', 'Thriller', 'War', 'Western']
items = pd.read_csv('ml-100k/u.item', sep='|', names=i_cols, encoding='latin-1')


# Top 5 recommendations
top_n = 5 
activeUser = 0 # Specific user to recommend to
mask_activeUser = ratings[activeUser, :] > 0

userAvgSort = np.where(~mask_activeUser)[0].argsort()
print('movie ID' + '\t movie title')
print(items['movie title'][userAvgSort[list(range(len(userAvgSort) - 1, len(userAvgSort) - top_n - 1, -1))]])

predicted_ds = np.repeat(userAvg.reshape(-1, 1), n_items, axis=1)

MAE_solution1 = evaluate(test_ds, predicted_ds)[0] # 0 is an intial value, you need to update this with the actual perofrmance of your implementation.
RMSE_solution1 = evaluate(test_ds, predicted_ds)[1] # 0 is an intial value, you need to update this with the actual perofrmance of your implementation.

movie ID	 movie title
1455          Beat the Devil (1954)
1454             Outlaw, The (1943)
1453    Angel and the Badman (1947)
1452    Angel on My Shoulder (1946)
1451       Lady of Burlesque (1943)
Name: movie title, dtype: object


In [6]:
# Please don't change this cell

print("===================== The MAE and RMSE of Your Implementation =====================")
print("MAE: {}, RMSE: {}" .format(MAE_solution1, RMSE_solution1))

===================== The MAE and RMSE of Your Implementation =====================
MAE: 0.8258905090105331, RMSE: 1.031143070595138


# Your Solution for Method 2

In [7]:
# Write your code here for Method 2
# You are required to implement the required solution 1 here. 
# Then, evaluate your implementation by predicting the ratings in the test set (test_ds).
# Finally, save the corresponding MAE and RMSE of your implementation 
# into the following defined corresponding variable. 

# Setting up parameters
itemRateNum = item_popularity
itemRateSum = train_ds.sum(axis=0)
itemRateAvg = itemRateSum/itemRateNum

#Implementation of MovieAvg to recommend the top_n = 5 movies to the activeUser = 0
top_n = 5
activeUser = 0 # Specific user to recommend to
mask_activeUser = ratings[activeUser, :] > 0
itemRateAvgCurrent = itemRateAvg.copy()
itemRateAvgCurrent[mask_activeUser] = 0

itemSortInd = itemRateAvgCurrent.argsort()
print('movie ID' + '\t movie title')
print(items['movie title'][itemSortInd[list(range(len(itemSortInd) - 1, len(itemSortInd) - top_n - 1, -1))]])

predicted_ds = np.repeat(itemRateAvg.reshape(1, -1), n_users, axis=0)



MAE_solution2 = evaluate(test_ds, predicted_ds)[0] # 0 is an intial value, you need to update this with the actual perofrmance of your implementation.
RMSE_solution2 = evaluate(test_ds, predicted_ds)[1] # 0 is an intial value, you need to update this with the actual perofrmance of your implementation.


movie ID	 movie title
1681    Scream of Stone (Schrei aus Stein) (1991)
1565              Man from Down Under, The (1943)
1647                      Niagara, Niagara (1997)
1491                       Window to Paris (1994)
1640                              Dadetown (1995)
Name: movie title, dtype: object


In [8]:
# Please don't change this cell

print("===================== The MAE and RMSE of Your Implementation =====================")
print("MAE: {}, RMSE: {}" .format(MAE_solution2, RMSE_solution2))

===================== The MAE and RMSE of Your Implementation =====================
MAE: 0.7961203950990416, RMSE: 1.0013142101585724


# Your Solution for Method 3

In [9]:
# Write your code here for Method 3
# You are required to implement the required solution 1 here. 
# Then, evaluate your implementation by predicting the ratings in the test set (test_ds).
# Finally, save the corresponding MAE and RMSE of your implementation 
# into the following defined corresponding variable. 


#Compute Pearson Correlation Coefficient for Each Pair of Users in Training Dataset
GAMMA = 30

train_ds = pd.DataFrame(train_ds)
np_user_pearson_corr = np.zeros((n_users, n_users))

for i, user_i_vec in enumerate(train_ds.values):
    for j, user_j_vec in enumerate(train_ds.values):
    
        # ratings corated by the current pair od users
        mask_i = user_i_vec > 0
        mask_j = user_j_vec > 0

        # corrated item index, skip if there are no corrated ratings
        corrated_index = np.intersect1d(np.where(mask_i), np.where(mask_j))
        if len(corrated_index) == 0:
            continue

        # average value of user_i_vec and user_j_vec
        mean_user_i = np.sum(user_i_vec) / (np.sum(np.clip(user_i_vec, 0, 1)) + EPSILON)
        mean_user_j = np.sum(user_j_vec) / (np.sum(np.clip(user_j_vec, 0, 1)) + EPSILON)

        # compute pearson corr
        user_i_sub_mean = user_i_vec[corrated_index] - mean_user_i
        user_j_sub_mean = user_j_vec[corrated_index] - mean_user_j

        r_ui_sub_r_i_sq = np.square(user_i_sub_mean)
        r_uj_sub_r_j_sq = np.square(user_j_sub_mean)

        r_ui_sum_sqrt = np.sqrt(np.sum(r_ui_sub_r_i_sq))
        r_uj_sum_sqrt = np.sqrt(np.sum(r_uj_sub_r_j_sq))

        sim = np.sum(user_i_sub_mean * user_j_sub_mean) / (r_ui_sum_sqrt * r_uj_sum_sqrt + EPSILON)

        # significance weighting
        weighted_sim = (min(len(corrated_index), GAMMA) / GAMMA) * sim

        np_user_pearson_corr[i][j] = weighted_sim



# Predict Ratings

np_predictions = np.zeros((n_users, n_items))
test_ds = pd.DataFrame(test_ds)

results = []
k_vals = [2, 5, 10, 15, 20, 25, 50, 100]

for K in k_vals:  # Finds the most accurate K value to optimise the algorithm
    for (i, j), rating in np.ndenumerate(test_ds.values):
        if rating > 0:
            # find top-k most similar users as the current user, remove itself
            sim_user_ids = np.argsort(np_user_pearson_corr[i])[-(K + 1):-1]

            # the coefficient values of similar users
            sim_val = np_user_pearson_corr[i][sim_user_ids]

            # the average value of the current user's ratings
            sim_users = train_ds.values[sim_user_ids]
            user_mean = np.sum(train_ds.values[i]) / (np.sum(np.clip(train_ds.values[i], 0, 1)) + EPSILON)
            sim_user_mean = np.sum(sim_users, axis=1) / (np.sum(np.clip(sim_users, 0, 1), axis=1) + EPSILON)

            # select the users who rated item j
            mask_rated_j = sim_users[:, j] > 0
            

    ## If all of the k nearest neighbours have no ratings from u_a, r_(a,i) will be predicted as the average rating of user u_a.
            if np.sum(mask_rated_j) == 0:
                np_predictions[i][j] = user_mean
            else:
            # sim(u, v) * (r_vj - mean_v)
                sim_r_sum_mean = sim_val[mask_rated_j] * (sim_users[mask_rated_j, j] - sim_user_mean[mask_rated_j])
                np_predictions[i][j] = user_mean + np.sum(sim_r_sum_mean) / (np.sum(sim_val[mask_rated_j]) + EPSILON)
                np_predictions[i][j] = np.clip(np_predictions[i][j], 0, 5)
    predicted_ds = pd.DataFrame(data=np_predictions) # Convert numpy array to pandas dataframe
    mae, rmse = evaluate(test_ds.values, predicted_ds.values)
    results.append({
        'K':      K,
        'MAE':    mae,
        'RMSE':   rmse
    })


df_results = pd.DataFrame(results) # Converts the results into a dataframe
print(df_results)
print(df_results.loc[df_results['MAE'].idxmin()]) # Minimum error at K = 100
print(df_results.loc[df_results['RMSE'].idxmin()])# Minimum error at K = 100


MAE_solution3 = df_results.iloc[7][1] # 0 is an intial value, you need to update this with the actual perofrmance of your implementation.
RMSE_solution3 = df_results.iloc[7][2] # 0 is an intial value, you need to update this with the actual perofrmance of your implementation.

     K       MAE      RMSE
0    2  0.852928  1.097084
1    5  0.839761  1.081909
2   10  0.807093  1.041305
3   15  0.784888  1.014519
4   20  0.770557  0.996595
5   25  0.759352  0.982060
6   50  0.735835  0.950393
7  100  0.724280  0.933518
K       100.000000
MAE       0.724280
RMSE      0.933518
Name: 7, dtype: float64
K       100.000000
MAE       0.724280
RMSE      0.933518
Name: 7, dtype: float64


In [10]:
# Please don't change this cell

print("===================== The MAE and RMSE of Your Implementation =====================")
print("MAE: {}, RMSE: {}" .format(MAE_solution3, RMSE_solution3))

===================== The MAE and RMSE of Your Implementation =====================
MAE: 0.7242796715862664, RMSE: 0.9335175303048852


# Your Solution for Method 4

In [11]:
# Write your code here for Method 4
# You are required to implement the required solution 1 here. 
# Then, evaluate your implementation by predicting the ratings in the test set (test_ds).
# Finally, save the corresponding MAE and RMSE of your implementation 
# into the following defined corresponding variable. 

DELTA = GAMMA

np_item_pearson_corr = np.zeros((n_items, n_items))
for i, item_i_vec in enumerate(train_ds.T.values):
    for j, item_j_vec in enumerate(train_ds.T.values):

        # ratings corated by the current pair od items
        mask_i = item_i_vec > 0
        mask_j = item_j_vec > 0

        # corrated index, impute corrated as item average if 
        # all of the k nearest neighbours have no ratings 
        corrated_index = np.intersect1d(np.where(mask_i), np.where(mask_j))
        if len(corrated_index) == 0:
            continue


        # average value of item_i_vec and item_j_vec
        mean_item_i = np.sum(item_i_vec) / (np.sum(np.clip(item_i_vec, 0, 1)) + EPSILON)
        mean_item_j = np.sum(item_j_vec) / (np.sum(np.clip(item_j_vec, 0, 1)) + EPSILON)

        # compute pearson corr
        item_i_sub_mean = item_i_vec[corrated_index] - mean_item_i
        item_j_sub_mean = item_j_vec[corrated_index] - mean_item_j

        r_ui_sub_ri_sq = np.square(item_i_sub_mean)
        r_uj_sub_rj_sq = np.square(item_j_sub_mean)

        r_ui_sub_ri_sq_sum_sqrt = np.sqrt(np.sum(r_ui_sub_ri_sq))
        r_uj_sub_rj_sq_sum_sqrt = np.sqrt(np.sum(r_uj_sub_rj_sq))

        sim = np.sum(item_i_sub_mean * item_j_sub_mean) / (r_ui_sub_ri_sq_sum_sqrt * r_uj_sub_rj_sq_sum_sqrt + EPSILON)

        # significance weighting
        weighted_sim = (min(len(corrated_index), DELTA) / DELTA) * sim

        np_item_pearson_corr[i][j] = weighted_sim


np_predictions = np.zeros((n_users, n_items))
results = []

k_vals = [2, 5, 10, 15, 20, 25, 50, 100] # options of k values 
for K in k_vals:  # Finds the most accurate K value to optimise the algorithm
    for (i, j), rating in np.ndenumerate(test_ds.values):
        if rating > 0:
            # find top-k most similar items as the current item, remove itself
            sim_item_ids = np.argsort(np_item_pearson_corr[j])[-(K + 1):-1]

            # the coefficient values of similar items
            sim_val = np_item_pearson_corr[j][sim_item_ids]

            # the average value of the current item's ratings
            sim_items = train_ds.T.values[sim_item_ids]
            item_mean = np.sum(train_ds.T.values[j]) / (np.sum(np.clip(train_ds.T.values[j], 0, 1)) + EPSILON)
            sim_item_mean = np.sum(sim_items, axis=1) / (np.sum(np.clip(sim_items, 0, 1), axis=1) + EPSILON)

            if np.sum(sim_val) ==0:  ## Fall back if all of the k nearest neighbours have no ratings 
                np_predictions[i][j] = item_mean # Imputes Item Mean
            else: # Otherwise use Item KNN-based Collaborative Filtering 

            # sim(u, v) * (r_v - mean_v)
                sim_r_sum_mean = sim_val * (sim_items[:, i] - sim_item_mean) 

            # filter unrated items
                w = np.clip(sim_items[:, i], 0, 1)
                sim_r_sum_mean *= w

                np_predictions[i][j] = item_mean + np.sum(sim_r_sum_mean) / (np.sum(sim_val * w) + EPSILON)    
                np_predictions[i][j] = np.clip(np_predictions[i][j], 0, 5)
 
    predicted_ds = pd.DataFrame(data=np_predictions) # Convert numpy array to pandas dataframe
    mae, rmse = evaluate(test_ds.values, predicted_ds.values)
    results.append({'K': K, 'MAE': mae, 'RMSE': rmse})
    
df_results = pd.DataFrame(results) # Converts the results into a dataframe
print(df_results)
print(df_results.loc[df_results['MAE'].idxmin()]) # Minimum error at K = 100
print(df_results.loc[df_results['RMSE'].idxmin()])# Minimum error at K = 100




MAE_solution4 = df_results.iloc[7][1] # 0 is an intial value, you need to update this with the actual perofrmance of your implementation.
RMSE_solution4 = df_results.iloc[7][2] # 0 is an intial value, you need to update this with the actual perofrmance of your implementation.


     K       MAE      RMSE
0    2  0.826346  1.070086
1    5  0.811873  1.055337
2   10  0.785626  1.022690
3   15  0.765403  0.993999
4   20  0.754664  0.978483
5   25  0.746116  0.965538
6   50  0.725046  0.932610
7  100  0.711640  0.913458
K       100.000000
MAE       0.711640
RMSE      0.913458
Name: 7, dtype: float64
K       100.000000
MAE       0.711640
RMSE      0.913458
Name: 7, dtype: float64


In [12]:
# Please don't change this cell

print("===================== The MAE and RMSE of Your Implementation =====================")
print("MAE: {}, RMSE: {}" .format(MAE_solution4, RMSE_solution4))

===================== The MAE and RMSE of Your Implementation =====================
MAE: 0.7116404641637847, RMSE: 0.9134583532587439


# Your Solution for Method 5

In [13]:
# Write your code here for Method 5
# You are required to implement the required solution 1 here. 
# Then, evaluate your implementation by predicting the ratings in the test set (test_ds).
# Finally, save the corresponding MAE and RMSE of your implementation 
# into the following defined corresponding variable. 


K = 100 # this can be said as both method 3 and 4 have optimal K values at K=100
np_predictions_item5 = np.zeros((n_users, n_items))

## Item based Knn Collaborative filtering
for (i, j), rating in np.ndenumerate(test_ds.values):
    if rating > 0:
            # find top-k most similar items as the current item, remove itself
        sim_item_ids = np.argsort(np_item_pearson_corr[j])[-(K + 1):-1]

            # the coefficient values of similar items
        sim_val = np_item_pearson_corr[j][sim_item_ids]

            # the average value of the current item's ratings
        sim_items = train_ds.T.values[sim_item_ids]
        item_mean = np.sum(train_ds.T.values[j]) / (np.sum(np.clip(train_ds.T.values[j], 0, 1)) + EPSILON)
        sim_item_mean = np.sum(sim_items, axis=1) / (np.sum(np.clip(sim_items, 0, 1), axis=1) + EPSILON)



            # sim(u, v) * (r_v - mean_v)
        sim_r_sum_mean = sim_val * (sim_items[:, i] - sim_item_mean) 

            # filter unrated items
        w = np.clip(sim_items[:, i], 0, 1)
        sim_r_sum_mean *= w

        np_predictions_item5[i][j] = item_mean + np.sum(sim_r_sum_mean) / (np.sum(sim_val * w) + EPSILON)    
        np_predictions_item5[i][j] = np.clip(np_predictions_item5[i][j], 0, 5)


## User Based Knn Collaborative Filtering 
np_predictions_user5 = np.zeros((n_users, n_items))
for (i, j), rating in np.ndenumerate(test_ds.values):
    if rating > 0:
        # find top-k most similar users as the current user, remove itself
        sim_user_ids = np.argsort(np_user_pearson_corr[i])[-(K + 1):-1]

        # the coefficient values of similar users
        sim_val = np_user_pearson_corr[i][sim_user_ids]

        # the average value of the current user's ratings
        sim_users = train_ds.values[sim_user_ids]
        user_mean = np.sum(train_ds.values[i]) / (np.sum(np.clip(train_ds.values[i], 0, 1)) + EPSILON)
        sim_user_mean = np.sum(sim_users, axis=1) / (np.sum(np.clip(sim_users, 0, 1), axis=1) + EPSILON)

        # select the users who rated item j
        mask_rated_j = sim_users[:, j] > 0
    
        # sim(u, v) * (r_vj - mean_v)
        sim_r_sum_mean = sim_val[mask_rated_j] * (sim_users[mask_rated_j, j] - sim_user_mean[mask_rated_j])
        np_predictions_user5[i][j] = user_mean + np.sum(sim_r_sum_mean) / (np.sum(sim_val[mask_rated_j]) + EPSILON)
        np_predictions_user5[i][j] = np.clip(np_predictions_user5[i][j], 0, 5)



## Finding optimal lambda!
lambda_results = []
λ_val = np.linspace(0,1,51) # Generates 50 values spread at an even interval throughout the range of 0 to 1
for λ in λ_val:
    np_predictions_5 = λ*np_predictions_user5 + (1-λ)*np_predictions_item5
    predicted_ds_5 = pd.DataFrame(data=np_predictions_5) # Convert numpy array to pandas dataframe
    mae, rmse = evaluate(test_ds.values, predicted_ds_5.values)
    lambda_results.append({'MAE': mae, 'RMSE': rmse})

lambda_results = pd.DataFrame(lambda_results) # Converts the results into a dataframe
#print(lambda_results)
print(lambda_results.loc[df_results['MAE'].idxmin()]) # The Minimum Mean absolute error at index 7
print(lambda_results.loc[df_results['RMSE'].idxmin()]) # The Minimum Root Mean Square Error at index 7
print(lambda_results.iloc[7]) # Minimum error at index 7
print('Optimal Value Of Lambda', λ_val[7])



MAE_solution5 = lambda_results.iloc[7][0] # 0 is an intial value, you need to update this with the actual perofrmance of your implementation.
RMSE_solution5 = lambda_results.iloc[7][1] # 0 is an intial value, you need to update this with the actual perofrmance of your implementation.


MAE     0.708552
RMSE    0.909185
Name: 7, dtype: float64
MAE     0.708552
RMSE    0.909185
Name: 7, dtype: float64
MAE     0.708552
RMSE    0.909185
Name: 7, dtype: float64
Optimal Value Of Lambda 0.14


In [14]:
# Please don't change this cell

print("===================== The MAE and RMSE of Your Implementation =====================")
print("MAE: {}, RMSE: {}" .format(MAE_solution5, RMSE_solution5))

===================== The MAE and RMSE of Your Implementation =====================
MAE: 0.7085519671646524, RMSE: 0.909185290610351
